In [2]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('input/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]


### XGBoost
### LightGBM

In [4]:

# -----------------------------------
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)

[0]	train-error:0.128533	eval-error:0.1516
[1]	train-error:0.115333	eval-error:0.146
[2]	train-error:0.109333	eval-error:0.1376
[3]	train-error:0.105333	eval-error:0.1364
[4]	train-error:0.096933	eval-error:0.1384
[5]	train-error:0.094667	eval-error:0.1364
[6]	train-error:0.087333	eval-error:0.1296
[7]	train-error:0.084933	eval-error:0.1244


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[8]	train-error:0.078133	eval-error:0.1208
[9]	train-error:0.073733	eval-error:0.1172
[10]	train-error:0.068667	eval-error:0.116
[11]	train-error:0.064933	eval-error:0.1164
[12]	train-error:0.062267	eval-error:0.1112
[13]	train-error:0.060533	eval-error:0.1116
[14]	train-error:0.0568	eval-error:0.1112
[15]	train-error:0.0504	eval-error:0.1048
[16]	train-error:0.0492	eval-error:0.1004
[17]	train-error:0.0464	eval-error:0.1016
[18]	train-error:0.044267	eval-error:0.1028
[19]	train-error:0.043467	eval-error:0.1012
[20]	train-error:0.038667	eval-error:0.1016
[21]	train-error:0.036533	eval-error:0.1004
[22]	train-error:0.034933	eval-error:0.0996
[23]	train-error:0.033733	eval-error:0.1004
[24]	train-error:0.032533	eval-error:0.104
[25]	train-error:0.0312	eval-error:0.1032
[26]	train-error:0.0288	eval-error:0.102
[27]	train-error:0.027733	eval-error:0.1016
[28]	train-error:0.025733	eval-error:0.1004
[29]	train-error:0.0232	eval-error:0.1008
[30]	train-error:0.022933	eval-error:0.1024
[31]	tr

In [5]:

# -----------------------------------
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71,
          'eval_metric': 'logloss'}
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)

# 最適な決定木の本数で予測を行う
pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)


[0]	train-logloss:0.540884	eval-logloss:0.550034
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.452691	eval-logloss:0.471821
[2]	train-logloss:0.39482	eval-logloss:0.420259
[3]	train-logloss:0.351978	eval-logloss:0.385202
[4]	train-logloss:0.320213	eval-logloss:0.361498
[5]	train-logloss:0.29673	eval-logloss:0.344634
[6]	train-logloss:0.276105	eval-logloss:0.329003
[7]	train-logloss:0.258858	eval-logloss:0.316697
[8]	train-logloss:0.243629	eval-logloss:0.307749
[9]	train-logloss:0.231528	eval-logloss:0.300925
[10]	train-logloss:0.220163	eval-logloss:0.294131
[11]	train-logloss:0.209625	eval-logloss:0.285282
[12]	train-logloss:0.199507	eval-logloss:0.279123
[13]	train-logloss:0.193238	eval-logloss:0.276415
[14]	train-logloss:0.185473	eval-logloss:0.271543
[15]	train-logloss:0.174737	eval-logloss:0.265163
[16]	train-logloss:0.168997	eval-logloss:0.260891
[17]	train-lo

In [8]:
# -----------------------------------
# lightgbmの実装
# -----------------------------------
import lightgbm as lgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をlightgbmのデータ構造に変換する
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

# ハイパーパラメータの設定
params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
num_round = 100

# 学習の実行
# カテゴリ変数をパラメータで指定している
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
categorical_features = ['product', 'medical_info_b2', 'medical_info_b3']
model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  categorical_feature=categorical_features,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['medical_info_b2', 'medical_info_b3', 'product']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	train's binary_logloss: 0.454286	valid's binary_logloss: 0.4654
[2]	train's binary_logloss: 0.429348	valid's binary_logloss: 0.443537
[3]	train's binary_logloss: 0.409269	valid's binary_logloss: 0.425588
[4]	train's binary_logloss: 0.393109	valid's binary_logloss: 0.411213
[5]	train's binary_logloss: 0.379351	valid's binary_logloss: 0.399341
[6]	train's binary_logloss: 0.366138	valid's binary_logloss: 0.389055
[7]	train's binary_logloss: 0.35417	valid's binary_logloss: 0.378254
[8]	train's binary_logloss: 0.343782	valid's binary_logloss: 0.370131
[9]	train's binary_logloss: 0.334283	valid's binary_logloss: 0.362036
[10]	train's binary_logloss: 0.324802	valid's binary_logloss: 0.353452
[11]	train's binary_logloss: 0.316592	valid's binary_logloss: 0.346904
[1

In [7]:
!pip install lightgbm